# [TUTORIAL] Constrained Task Planning (Sweep)

* This tutorial is written to instruct basic usage of the task & motion planning pipeline  
* Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  

## add project source directory to path

In [1]:
import os
import sys
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7gripper, None,
                INDY_IP)]
              , connection_list=[False])

connection command:
indy0: False


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# # deprecated: s_builder.reset_reference_coord(ref_name="floor")

## create GeometryScene

In [4]:
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## init planning scene

In [8]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.4,-0.4,0), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)

track = gscene.create_safe(GEOTYPE.BOX, "track", "base_link", (0.1,0.5,0.01), (0.5,0,0), rpy=(0,0,0), 
                           color=(1,0.7,0.7,0.5), display=True, fixed=True, collision=False, parent="floor")
wp1 = gscene.create_safe(GEOTYPE.BOX, "wp1", "base_link", (0.1,0.1,0.01), (0, 0.2,0.),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="track")
wp2 = gscene.create_safe(GEOTYPE.BOX, "wp2", "base_link", (0.1,0.1,0.01), (0, -0.2,0), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,1), display=True, fixed=True, collision=False, parent="track")

Please create a subscriber to the marker


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

## add brush
* brush is a CustomObject with one grasping handle and one placement point on the bottom face

In [11]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint

brush_body = gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.2,0.07,0.02), 
                   center=(0,0.5,0.045), rpy=(0,0,np.pi/2), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
brush_handle = gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.2,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
brush_face = gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.19,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

brush = pscene.create_subject(oname="brush", gname="brush_body", _type=CustomObject, 
                              action_points_dict={"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2, 0, 0]),
                                                  "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

## Register binders

In [12]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, PlaceFrame

In [13]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [14]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane, point=None)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))

### planning pipeline

In [15]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## ui

In [16]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## SweepTask (Plane constraint)
* **SweepTask** is a **WaypointTask** with plane constraint, where a **WaypointTask** is a task of sequentially touching defined waypoints
* With the plane constraint, the brush face maintains contact with the surface
* In this example the **SweepTask** contains two wayoints
* **SweepPoint** and **SweepTool** are action point and actor for sweep task, without fixed orientation

In [17]:
from pkg.planning.constraint.constraint_subject import SweepPoint, SweepTask
from pkg.planning.constraint.constraint_actor import SweepTool
sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])})
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))
ppline.set_task_planner(tplan)

### Planning

In [18]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

initial_state = pscene.initialize_state(crob.home_pose)

In [19]:
# goal_nodes = [("goal",)]
goal_nodes = [("goal", 2)]
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=60, multiprocess=False)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.2
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('grip0', 0) = success
branching: 0->1 (0.18/60.0 s, steps/err: 40(72.4930763245 ms)/0.00195549030136)
try: 1 - ('grip0', 0)->('floor', 0)
error: 2.8
try transition motion
transition motion tried: False
Motion Plan Failure
result: 1 - ('grip0', 0)->('floor', 0) = fail
try: 1 - ('grip0', 0)->('grip0',

result: 2 - ('grip0', 1)->('grip0', 2) = fail
try: 2 - ('grip0', 1)->('floor', 1)
error: 3.3
try transition motion
transition motion tried: False
Motion Plan Failure
result: 2 - ('grip0', 1)->('floor', 1) = fail
try: 7 - ('grip0', 0)->('goal', 0)
error: 1.5
try transition motion
transition motion tried: True
result: 7 - ('grip0', 0)->('goal', 0) = success
branching: 7->16 (11.78/60.0 s, steps/err: 88(86.6529941559 ms)/0.00139870565464)
try: 16 - ('goal', 0)->('goal', 1)
result: 16 - ('goal', 0)->('goal', 1) = fail
try: 1 - ('grip0', 0)->('goal', 0)
error: 2.7
try transition motion
transition motion tried: True
result: 1 - ('grip0', 0)->('goal', 0) = success
branching: 1->17 (11.87/60.0 s, steps/err: 54(72.4010467529 ms)/0.000747575873236)
try: 17 - ('goal', 0)->('goal', 1)
result: 17 - ('goal', 0)->('goal', 1) = fail
try: 7 - ('grip0', 0)->('goal', 0)
error: 2.8
try transition motion
transition motion tried: False
Motion Plan Failure
result: 7 - ('grip0', 0)->('goal', 0) = fail
try: 10

try: 24 - ('floor', 1)->('grip0', 1)
error: 0.12
try transition motion
transition motion tried: True
result: 24 - ('floor', 1)->('grip0', 1) = success
branching: 24->34 (22.92/60.0 s, steps/err: 4(40.0619506836 ms)/0.00181910227321)
try: 6 - ('floor', 0)->('floor', 1)
result: 6 - ('floor', 0)->('floor', 1) = fail
try: 7 - ('grip0', 0)->('floor', 0)
error: 2.6
try transition motion
transition motion tried: True
result: 7 - ('grip0', 0)->('floor', 0) = success
branching: 7->35 (23.0/60.0 s, steps/err: 29(65.4017925262 ms)/0.00120954449123)
try: 24 - ('floor', 1)->('grip0', 1)
error: 0.21
try transition motion
transition motion tried: True
result: 24 - ('floor', 1)->('grip0', 1) = success
branching: 24->36 (23.04/60.0 s, steps/err: 4(34.8868370056 ms)/0.000685362694052)
try: 25 - ('grip0', 2)->('floor', 2)
error: 2.1
try transition motion
transition motion tried: False
Motion Plan Failure
result: 25 - ('grip0', 2)->('floor', 2) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floo

error: 2.4
try transition motion
transition motion tried: False
Motion Plan Failure
result: 31 - ('grip0', 2)->('floor', 2) = fail
try: 39 - ('goal', 0)->('goal', 1)
result: 39 - ('goal', 0)->('goal', 1) = fail
try: 41 - ('grip0', 1)->('goal', 1)
error: 0.59
try transition motion
transition motion tried: True
result: 41 - ('grip0', 1)->('goal', 1) = success
branching: 41->50 (37.44/60.0 s, steps/err: 4(32.998085022 ms)/0.00134966714499)
try: 50 - ('goal', 1)->('goal', 2)
result: 50 - ('goal', 1)->('goal', 2) = fail
try: 25 - ('grip0', 2)->('floor', 2)
error: 2.0
try transition motion
transition motion tried: False
Motion Plan Failure
result: 25 - ('grip0', 2)->('floor', 2) = fail
try: 20 - ('grip0', 1)->('floor', 1)
error: 2.1
try transition motion
transition motion tried: True
result: 20 - ('grip0', 1)->('floor', 1) = success
branching: 20->51 (38.54/60.0 s, steps/err: 51(63.8589859009 ms)/0.00150764455409)
try: 39 - ('goal', 0)->('goal', 1)
result: 39 - ('goal', 0)->('goal', 1) = fai

result: 19 - ('grip0', 0)->('floor', 0) = fail
try: 31 - ('grip0', 2)->('goal', 2)
error: 1.2
try transition motion
transition motion tried: True
Goal reached
result: 31 - ('grip0', 2)->('goal', 2) = success
branching: 31->67 (46.16/60.0 s, steps/err: 74(106.507062912 ms)/0.00139371682407)
++ adding return motion to acquired answer ++
Goal reached
try: 42 - ('grip0', 0)->('goal', 0)
error: 0.78
try transition motion
transition motion tried: True
result: 42 - ('grip0', 0)->('goal', 0) = success
branching: 42->69 (46.46/60.0 s, steps/err: 12(53.7540912628 ms)/0.000980189413109)
try: 69 - ('goal', 0)->('goal', 1)
result: 69 - ('goal', 0)->('goal', 1) = fail
try: 16 - ('goal', 0)->('grip0', 0)
error: 1.5
try transition motion
transition motion tried: True
result: 16 - ('goal', 0)->('grip0', 0) = success
branching: 16->70 (46.62/60.0 s, steps/err: 76(139.991044998 ms)/0.00188130678928)
try: 63 - ('grip0', 1)->('floor', 1)
error: 2.3
try transition motion
transition motion tried: False
Motio

### play searched plan

In [21]:
ppline.play_schedule(snode_schedule, period=0.01)

('floor', 0)->('grip0', 0)
('grip0', 0)->('floor', 0)
('floor', 0)->('grip0', 0)
('grip0', 0)->('floor', 0)
('floor', 0)->('grip0', 0)
('grip0', 0)->('grip0', 1)
('grip0', 1)->('grip0', 2)
('grip0', 2)->('goal', 2)
('goal', 2)->('goal', 2)


### return to initial state to run next example

In [22]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

## SweepLineTask (Line constraint)
* **SweepLineTask** is a **SweepTask** with a line constraint
* A straight line constraint is generated between two given waypoints
* By default, **SweepLineTask** fixes orientation while sweeping.
* **SweepFrame** and **SweepFramer** are action point and actor for sweep task that with fixed orientations.
* To observe orientation constraint, **SweepFrame** and **SweepFramer** should be used.

In [23]:
from pkg.planning.constraint.constraint_subject import SweepLineTask, SweepFrame
from pkg.planning.constraint.constraint_actor import SweepFramer

sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepFrame("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepFrame("wp2", wp2, [0,0,0.005], [0,0,0])})
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,-0.015), rpy=(0,0,0))
ppline.set_task_planner(tplan)

### Planning

In [24]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

initial_state = pscene.initialize_state(crob.home_pose)

In [25]:
# goal_nodes = [("goal",)]
goal_nodes = [("goal", 2)]
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=60, multiprocess=False)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

try: 0 - ('floor', 0)->('grip0', 0)
error: 2.2
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('floor', 0)->('grip0', 0) = fail
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.2
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('floor', 0)->('grip0', 0) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.3
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('grip0', 0) = success
branching: 0->1 (2.15/60.0 s, steps/err: 52(61.6888999939 ms)/0.00174616978829)
try: 1 - ('grip0', 0)->('grip0', 1)
error: 2.1
try transition motion
transition motion tried: True
result: 1 - ('grip0', 0)->('grip0', 1) = success
branching: 1->2 (2.19/60.0 s, steps/err: 14(41.0640239716 ms)/0.00189810555179)
try: 1 - ('grip0', 0)->('grip0', 1)
error: 2.1
try transition motion
transition motion tried: True
result: 1 - ('grip0', 0)->('gri

Motion Plan Failure
result: 1 - ('grip0', 0)->('floor', 0) = fail
try: 19 - ('goal', 1)->('grip0', 1)
error: 0.95
try transition motion
transition motion tried: True
result: 19 - ('goal', 1)->('grip0', 1) = success
branching: 19->21 (12.83/60.0 s, steps/err: 40(52.4990558624 ms)/0.000447965098531)
try: 18 - ('floor', 0)->('floor', 1)
result: 18 - ('floor', 0)->('floor', 1) = fail
try: 1 - ('grip0', 0)->('floor', 0)
error: 2.2
try transition motion
transition motion tried: False
Motion Plan Failure
result: 1 - ('grip0', 0)->('floor', 0) = fail
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.3
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('floor', 0)->('grip0', 0) = fail
try: 1 - ('grip0', 0)->('grip0', 1)
error: 2.1
try transition motion
transition motion tried: True
result: 1 - ('grip0', 0)->('grip0', 1) = success
branching: 1->22 (14.96/60.0 s, steps/err: 14(47.2400188446 ms)/0.000813910824871)
try: 1 - ('grip0', 0)->('grip0', 1)
error: 2.1
try tra

transition motion tried: True
Goal reached
result: 36 - ('grip0', 2)->('goal', 2) = success
branching: 36->37 (23.53/60.0 s, steps/err: 51(79.6511173248 ms)/0.0012069362336)
++ adding return motion to acquired answer ++
Goal reached
try: 1 - ('grip0', 0)->('goal', 0)
error: 2.5
try transition motion
transition motion tried: True
result: 1 - ('grip0', 0)->('goal', 0) = success
branching: 1->39 (23.84/60.0 s, steps/err: 88(72.9911327362 ms)/0.00120769046585)
try: 39 - ('goal', 0)->('goal', 1)
result: 39 - ('goal', 0)->('goal', 1) = fail
try: 35 - ('floor', 2)->('grip0', 2)
error: 0.28
try transition motion
transition motion tried: True
result: 35 - ('floor', 2)->('grip0', 2) = success
branching: 35->40 (24.02/60.0 s, steps/err: 9(163.854837418 ms)/0.00104733923958)
try: 40 - ('grip0', 2)->('goal', 2)
error: 0.75
try transition motion
transition motion tried: True
Goal reached
result: 40 - ('grip0', 2)->('goal', 2) = success
branching: 40->41 (24.12/60.0 s, steps/err: 94(99.8811721802 ms)

### play searched plan

In [26]:
ppline.play_schedule(snode_schedule, period=0.01)

('floor', 0)->('grip0', 0)
('grip0', 0)->('grip0', 1)
('grip0', 1)->('grip0', 2)
('grip0', 2)->('goal', 2)
('goal', 2)->('goal', 2)


### return to initial state to run next example

In [27]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

## SweepLineTask without orientation constraint
* By setting ***SweepLineTask.fix_direction=False***, you can disable orientation consraint
* As the orientation constraint is disabled, you can use **SweepPoint** and **SweepTool** with **SweepLineTask**

In [28]:
from pkg.planning.constraint.constraint_subject import SweepLineTask, SweepFrame
from pkg.planning.constraint.constraint_actor import SweepFramer

sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepPoint("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepPoint("wp2", wp2, [0,0,0.005], [0,0,0])})
sweep.fix_direction = False # disable direction constraint
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))
ppline.set_task_planner(tplan)

### Planning

In [29]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

initial_state = pscene.initialize_state(crob.home_pose)

In [30]:
# goal_nodes = [("goal",)]
goal_nodes = [("goal", 2)]
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=60, multiprocess=False)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.2
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('floor', 0)->('grip0', 0) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.2
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('floor', 0)->('grip0', 0) = fail
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.2
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('grip0', 0) = success
branching: 0->1 (2.22/60.0 s, steps/err: 47(66.5121078491 ms)/0.00147599788236)
try: 1 - ('grip0', 0)->('floor

result: 12 - ('grip0', 1)->('grip0', 2) = fail
try: 10 - ('grip0', 0)->('floor', 0)
error: 2.3
try transition motion
transition motion tried: False
Motion Plan Failure
result: 10 - ('grip0', 0)->('floor', 0) = fail
try: 12 - ('grip0', 1)->('goal', 1)
error: 2.8
try transition motion
transition motion tried: False
Motion Plan Failure
result: 12 - ('grip0', 1)->('goal', 1) = fail
try: 10 - ('grip0', 0)->('goal', 0)
error: 1.1
try transition motion
transition motion tried: True
result: 10 - ('grip0', 0)->('goal', 0) = success
branching: 10->16 (12.72/60.0 s, steps/err: 16(52.0069599152 ms)/0.00173229149357)
try: 16 - ('goal', 0)->('goal', 1)
result: 16 - ('goal', 0)->('goal', 1) = fail
try: 14 - ('goal', 0)->('grip0', 0)
error: 1.9
try transition motion
transition motion tried: True
result: 14 - ('goal', 0)->('grip0', 0) = success
branching: 14->17 (12.81/60.0 s, steps/err: 78(74.0578174591 ms)/0.00143429992122)
try: 1 - ('grip0', 0)->('grip0', 1)
error: 2.3
try transition motion
transiti

result: 12 - ('grip0', 1)->('grip0', 2) = fail
try: 24 - ('grip0', 1)->('grip0', 2)
error: 1.8
try constrained motion
constrained motion tried: False
Motion Plan Failure
result: 24 - ('grip0', 1)->('grip0', 2) = fail
try: 27 - ('floor', 0)->('floor', 1)
result: 27 - ('floor', 0)->('floor', 1) = fail
try: 21 - ('goal', 0)->('grip0', 0)
error: 0.98
try transition motion
transition motion tried: True
result: 21 - ('goal', 0)->('grip0', 0) = success
branching: 21->33 (24.26/60.0 s, steps/err: 42(40.5838489532 ms)/0.00190258178016)
try: 22 - ('grip0', 1)->('floor', 1)
error: 1.0
try transition motion
transition motion tried: False
Motion Plan Failure
result: 22 - ('grip0', 1)->('floor', 1) = fail
try: 27 - ('floor', 0)->('grip0', 0)
error: 0.14
try transition motion
transition motion tried: False
Motion Plan Failure
result: 27 - ('floor', 0)->('grip0', 0) = fail
try: 22 - ('grip0', 1)->('floor', 1)
error: 3.1
try transition motion
transition motion tried: False
Motion Plan Failure
result: 2

transition motion tried: True
result: 27 - ('floor', 0)->('grip0', 0) = success
branching: 27->47 (35.81/60.0 s, steps/err: 104(128.012895584 ms)/0.00132587960179)
try: 37 - ('goal', 1)->('goal', 2)
result: 37 - ('goal', 1)->('goal', 2) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 36 - ('goal', 1)->('grip0', 1)
error: 0.59
try transition motion
transition motion tried: True
result: 36 - ('goal', 1)->('grip0', 1) = success
branching: 36->48 (35.88/60.0 s, steps/err: 15(49.6909618378 ms)/0.0012443621061)
try: 27 - ('floor', 0)->('floor', 1)
result: 27 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.2
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('grip0', 0) = success
branching: 0->49 (35.96/60.0 s, steps/err: 58(70.4009532928 ms)/0.0010942131105)
try: 39 - ('floor', 1)->('floor', 2)
result: 39 - ('floor', 1)->('floor', 2) = fail
try: 8 - ('goal', 1)->('goal', 2)
result: 8 - 

transition motion tried: True
result: 52 - ('floor', 1)->('grip0', 1) = success
branching: 52->68 (42.4/60.0 s, steps/err: 80(93.593120575 ms)/0.00198311307554)
try: 53 - ('goal', 1)->('grip0', 1)
error: 1.4
try transition motion
transition motion tried: True
result: 53 - ('goal', 1)->('grip0', 1) = success
branching: 53->69 (42.47/60.0 s, steps/err: 70(71.2580680847 ms)/0.0015502779116)
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.2
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('grip0', 0) = success
branching: 0->70 (42.56/60.0 s, steps/err: 49(88.2239341736 ms)/0.00127514711277)
try: 35 - ('goal', 0)->('goal', 1)
result: 35 - ('goal', 0)->('goal', 1) = fail
try: 23 - ('grip0', 0)->('floor', 0)
error: 2.7
try transition motion
transition motion tried: False
Motion Plan Failure
result: 23 - ('grip0', 0)->('floor', 0) = fail
try: 28 - ('grip0', 0)->('floor', 0)
error: 2.5
try transition motion
transition motion tried: False
Motion Plan Failure
result: 28

### play searched plan

In [31]:
ppline.play_schedule(snode_schedule, period=0.01)

('floor', 0)->('grip0', 0)
('grip0', 0)->('goal', 0)
('goal', 0)->('grip0', 0)
('grip0', 0)->('goal', 0)
('goal', 0)->('grip0', 0)
('grip0', 0)->('grip0', 1)
('grip0', 1)->('grip0', 2)
('grip0', 2)->('goal', 2)
('goal', 2)->('goal', 2)


### return to initial state to run next example

In [32]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

## SweepLineTask (mimic constrained planning by task-space line interpolation)
* Unfortunately, constrained motion planning shows very poor performance in many cases.
* In many cases, we can just mimic constrained planning by line interpolation in task-space
* ***MoveitPlanner.incremental_constraint_motion=True*** is the option that replaces constrained planning with task-space interpolation

In [33]:
from pkg.planning.constraint.constraint_subject import SweepLineTask, SweepFrame
from pkg.planning.constraint.constraint_actor import SweepFramer

sweep = pscene.create_subject(oname="sweep", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp1": SweepFrame("wp1", wp1, [0,0,0.005], [0,0,0]),
                                                   "wp2": SweepFrame("wp2", wp2, [0,0,0.005], [0,0,0])})
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepFramer, point=(0,0,-0.015), rpy=(0,0,0))
ppline.set_task_planner(tplan)
mplan.incremental_constraint_motion = True # this option enables line 

### Planning

In [34]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy

initial_state = pscene.initialize_state(crob.home_pose)

In [35]:
# goal_nodes = [("goal",)]
goal_nodes = [("goal", 2)]
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=60, multiprocess=False)
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.2
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('floor', 0)->('grip0', 0) = fail
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.2
try transition motion
transition motion tried: False
Motion Plan Failure
result: 0 - ('floor', 0)->('grip0', 0) = fail
try: 0 - ('floor', 0)->('grip0', 0)
error: 2.2
try transition motion
transition motion tried: True
result: 0 - ('floor', 0)->('grip0', 0) = success
branching: 0->1 (2.22/60.0 s, steps/err: 38(90.2218818665 ms)/0.00134524563082)
try: 1 - ('grip0', 0)->('floor', 0)
error: 3.3
try transition motion
transition motion tried: False
Motion Plan 

Goal reached
try: 1 - ('grip0', 0)->('floor', 0)
error: 3.0
try transition motion
transition motion tried: False
Motion Plan Failure
result: 1 - ('grip0', 0)->('floor', 0) = fail
try: 0 - ('floor', 0)->('floor', 1)
result: 0 - ('floor', 0)->('floor', 1) = fail
try: 1 - ('grip0', 0)->('floor', 0)
error: 1.8
try transition motion
transition motion tried: False
Motion Plan Failure
result: 1 - ('grip0', 0)->('floor', 0) = fail
try: 1 - ('grip0', 0)->('goal', 0)
error: 2.8
try transition motion
transition motion tried: True
result: 1 - ('grip0', 0)->('goal', 0) = success
branching: 1->22 (10.84/60.0 s, steps/err: 54(70.9149837494 ms)/0.00178991837502)
try: 22 - ('goal', 0)->('goal', 1)
result: 22 - ('goal', 0)->('goal', 1) = fail
try: 8 - ('grip0', 1)->('grip0', 2)
error: 0.4
end
constrained motion tried: True
result: 8 - ('grip0', 1)->('grip0', 2) = success
branching: 8->23 (11.0/60.0 s, steps/err: 41(133.592128754 ms)/0.000116594592284)
try: 23 - ('grip0', 2)->('goal', 2)
error: 2.8
try t

### play searched plan

In [36]:
ppline.play_schedule(snode_schedule, period=0.01)

('floor', 0)->('grip0', 0)
('grip0', 0)->('grip0', 1)
('grip0', 1)->('grip0', 2)
('grip0', 2)->('goal', 2)
('goal', 2)->('goal', 2)


### return to initial state

In [37]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)